# Ex2 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.

### Step 1. Import the necessary libraries

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.window import Window as W

In [2]:
spark = SparkSession.builder.appName("exercise2").getOrCreate()

23/01/23 19:07:34 WARN Utils: Your hostname, Ana-Matebook resolves to a loopback address: 127.0.1.1; using 192.168.1.137 instead (on interface wlp2s0)
23/01/23 19:07:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/23 19:07:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/23 19:07:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv). 

### Step 3. Assign it to a variable called euro12.

In [3]:
euro_df = spark.read.option("header", True).option("inferSchema", True).csv("Euro_2012_stats_TEAM.csv")

In [8]:
euro_df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Goals: integer (nullable = true)
 |-- Shots on target: integer (nullable = true)
 |-- Shots off target: integer (nullable = true)
 |-- Shooting Accuracy: string (nullable = true)
 |-- % Goals-to-shots: string (nullable = true)
 |-- Total shots (inc. Blocked): integer (nullable = true)
 |-- Hit Woodwork: integer (nullable = true)
 |-- Penalty goals: integer (nullable = true)
 |-- Penalties not scored: integer (nullable = true)
 |-- Headed goals: integer (nullable = true)
 |-- Passes: integer (nullable = true)
 |-- Passes completed: integer (nullable = true)
 |-- Passing Accuracy: string (nullable = true)
 |-- Touches: integer (nullable = true)
 |-- Crosses: integer (nullable = true)
 |-- Dribbles: integer (nullable = true)
 |-- Corners Taken: integer (nullable = true)
 |-- Tackles: integer (nullable = true)
 |-- Clearances: integer (nullable = true)
 |-- Interceptions: integer (nullable = true)
 |-- Clearances off line: integer (nullable = t

### Step 4. Select only the Goal column.

In [11]:
goal_df = euro_df.select("Goals")
goal_df.show(n=5)

+-----+
|Goals|
+-----+
|    4|
|    4|
|    4|
|    5|
|    3|
+-----+
only showing top 5 rows



### Step 5. How many team participated in the Euro2012?

In [14]:
euro_df.agg(func.count_distinct("Team")).show()

+-----------+
|count(Team)|
+-----------+
|         16|
+-----------+



### Step 6. What is the number of columns in the dataset?

In [15]:
len(euro_df.columns)

35

### Step 7. View only the columns Team, Yellow Cards and Red Cards and assign them to a dataframe called discipline

In [16]:
discipline = euro_df.select("Team", "Yellow Cards", "Red Cards")

In [17]:
discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|            Denmark|           4|        0|
|            England|           5|        0|
|             France|           6|        0|
|            Germany|           4|        0|
|             Greece|           9|        1|
|              Italy|          16|        0|
|        Netherlands|           5|        0|
|             Poland|           7|        1|
|           Portugal|          12|        0|
|Republic of Ireland|           6|        1|
|             Russia|           6|        0|
|              Spain|          11|        0|
|             Sweden|           7|        0|
|            Ukraine|           5|        0|
+-------------------+------------+---------+



### Step 8. Sort the teams by Red Cards, then to Yellow Cards

In [22]:
discipline = discipline.sort(["Red Cards", "Yellow Cards"], ascending=False)
discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|             Greece|           9|        1|
|             Poland|           7|        1|
|Republic of Ireland|           6|        1|
|              Italy|          16|        0|
|           Portugal|          12|        0|
|              Spain|          11|        0|
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|             Sweden|           7|        0|
|             Russia|           6|        0|
|             France|           6|        0|
|            England|           5|        0|
|        Netherlands|           5|        0|
|            Ukraine|           5|        0|
|            Denmark|           4|        0|
|            Germany|           4|        0|
+-------------------+------------+---------+



### Step 9. Calculate the mean Yellow Cards given per Team

In [24]:
discipline.groupBy("Team").agg(func.avg("Yellow Cards").alias("avg_yellow_cards")).show()

+-------------------+----------------+
|               Team|avg_yellow_cards|
+-------------------+----------------+
|             Russia|             6.0|
|             Sweden|             7.0|
|            Germany|             4.0|
|             France|             6.0|
|             Greece|             9.0|
|            Croatia|             9.0|
|              Italy|            16.0|
|              Spain|            11.0|
|            Denmark|             4.0|
|            Ukraine|             5.0|
|     Czech Republic|             7.0|
|Republic of Ireland|             6.0|
|            England|             5.0|
|             Poland|             7.0|
|           Portugal|            12.0|
|        Netherlands|             5.0|
+-------------------+----------------+



### Step 10. Filter teams that scored more than 6 goals

In [28]:
euro_df.filter(euro_df.Goals > 6).select("Team", "Goals").show()

+-------+-----+
|   Team|Goals|
+-------+-----+
|Germany|   10|
|  Spain|   12|
+-------+-----+



### Step 11. Select the teams that start with G

In [29]:
euro_df.filter(euro_df.Team.startswith("G")).select("Team").show()

+-------+
|   Team|
+-------+
|Germany|
| Greece|
+-------+



### Step 12. Select the first 7 columns

In [60]:
# There is some issue with the name of this column so we rename it
euro_df = euro_df.withColumnRenamed("Total shots (inc. Blocked)", "TotalShortsIncBlocked")

In [62]:
euro_df.select([x for i, x in enumerate(euro_df.columns) if i < 7]).show()

+-------------------+-----+---------------+----------------+-----------------+----------------+---------------------+
|               Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|TotalShortsIncBlocked|
+-------------------+-----+---------------+----------------+-----------------+----------------+---------------------+
|            Croatia|    4|             13|              12|            51.9%|           16.0%|                   32|
|     Czech Republic|    4|             13|              18|            41.9%|           12.9%|                   39|
|            Denmark|    4|             10|              10|            50.0%|           20.0%|                   27|
|            England|    5|             11|              18|            50.0%|           17.2%|                   40|
|             France|    3|             22|              24|            37.9%|            6.5%|                   65|
|            Germany|   10|             32|             

### Step 13. Select all columns except the last 3.

In [64]:
total_columns = len(euro_df.columns)
selected_euro_df = euro_df.select([x for i, x in enumerate(euro_df.columns) if (i < total_columns - 3)])

### Step 14. Present only the Shooting Accuracy from England, Italy and Russia

In [72]:
countries = ["England", "Italy", "Russia"]
euro_df.filter(euro_df.Team.isin(countries)).select("Team", "Goals").show()

+-------+-----+
|   Team|Goals|
+-------+-----+
|England|    5|
|  Italy|    6|
| Russia|    5|
+-------+-----+

